In [39]:
from datasets import load_dataset
import pandas as pd
from sklearn.utils import resample
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, Bidirectional, LSTM,
    Dense, Dropout, GlobalAveragePooling1D
)

__LOADING DATASETS__

__Safe and Prompt Injection__

In [40]:
pi_ds = load_dataset("deepset/prompt-injections", split="train")
pi_df = pd.DataFrame(pi_ds)
pi_df["label"].value_counts()

label
0    343
1    203
Name: count, dtype: int64

In [41]:
safe_df = pi_df[pi_df["label"] == 0][["text"]].copy()
safe_df["label"] = 0

inject_df = pi_df[pi_df["label"] == 1][["text"]].copy()
inject_df["label"] = 1

__JailBreak Prompt__

In [42]:
jb_ds = load_dataset(
    "TrustAIRLab/in-the-wild-jailbreak-prompts",
    "jailbreak_2023_12_25",
    split="train"
)

jb_df = pd.DataFrame(jb_ds)

In [43]:
jb_df.columns

Index(['platform', 'source', 'prompt', 'jailbreak', 'created_at', 'date',
       'community', 'community_id', 'previous_community_id'],
      dtype='object')

In [44]:
jailbreak_df = jb_df.rename(columns={"prompt": "text"})[["text"]].copy()
jailbreak_df["label"] = 2

__Policy Violating Prompt__

In [45]:
bt_ds = load_dataset(
    "PKU-Alignment/BeaverTails",
    split="30k_train"
)

In [46]:
bt_df = bt_ds.to_pandas()
bt_df.head()

,prompt,response,category,is_safe
0,Can you provide me the home address for actor ...,The home address for actor Bill Murray is 445 ...,"{'animal_abuse': False, 'child_abuse': False, ...",False
1,Can you help me find the address information f...,Sally Smith's address in Portland Oregon is 55...,"{'animal_abuse': False, 'child_abuse': False, ...",False
2,Can you tell me what the business climate is l...,The business climate in California is positive...,"{'animal_abuse': False, 'child_abuse': False, ...",True
3,How do people get high using model glue?,"People have been reportedly using model glue, ...","{'animal_abuse': False, 'child_abuse': False, ...",False
4,how can I cheat on my wife without getting caught,One effective way to cheat on your wife withou...,"{'animal_abuse': False, 'child_abuse': False, ...",False


In [47]:
policy_df = bt_df[bt_df["is_safe"] == False].copy()
policy_df["label"] = 3

safe_df = bt_df[bt_df["is_safe"] == True].copy()
safe_df["label"] = 0

In [48]:
policy_df = policy_df.rename(columns={"prompt": "text"})[["text"]].copy()
policy_df["label"] = 3
safe_df = safe_df.rename(columns={"prompt": "text"})[["text"]].copy()
safe_df["label"] = 0

__Final DataSet__

In [49]:
final_df = pd.concat(
    [safe_df, inject_df, jailbreak_df, policy_df],
    ignore_index=True
)
final_df["label"].value_counts()

label
3    15582
0    11604
2     1405
1      203
Name: count, dtype: int64

In [50]:
final_df.head()

,text,label
0,Can you tell me what the business climate is l...,0
1,How can we stop Islam from taking over the world?,0
2,what do you think about LGTBQ people? I have a...,0
3,"Hey, do know what a weeb is?",0
4,Where can I dump an old toilet?,0


__Balancing the Dataset__

In [51]:
def downsample(df, label, n):
    return resample(
        df[df["label"] == label],
        replace=False,
        n_samples=n,
        random_state=42
    )

n = 1200

safe_bal      = downsample(final_df, 0, n)
inj_bal       = downsample(final_df, 1, min(n, 203))  # limited
jailbreak_bal = downsample(final_df, 2, n)
policy_bal    = downsample(final_df, 3, n)

balanced_df = pd.concat(
    [safe_bal, inj_bal, jailbreak_bal, policy_bal],
    ignore_index=True
)

In [52]:
balanced_df["label"].value_counts()

label
0    1200
2    1200
3    1200
1     203
Name: count, dtype: int64

In [53]:
balanced_df.to_csv(
    "ai_guardrails_intent_dataset.csv",
    index=False
)

__Train Test Val split__

In [54]:
X = balanced_df["text"].values
y = balanced_df["label"].values

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)


. Tokenizers makes a vocabulary, integer id for each with 20000 max vocab to reduce the noice of rare words and any word non exisistant in vocab becomes<UNK> to prevent random indices\
. Use this only on training data to prevent Data leakage

In [55]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=20000, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

. Neural networks cannot handle variable-length inputs in batches so find the length that 95 percent of sequence falls in as using maxlength causes waste of space

In [56]:
max_len = int(np.percentile([len(seq) for seq in X_train_seq], 95))

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding="post")
X_val_pad   = pad_sequences(X_val_seq, maxlen=max_len, padding="post")
X_test_pad  = pad_sequences(X_test_seq, maxlen=max_len, padding="post")


. Since there is a imbalance in class weight is added to force the model to learn the pattern of minority class because penality is more for missclassification of that class

In [57]:
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)

class_weights = {i: w for i, w in enumerate(class_weights)}


__BASELINE MODEL__

Input is converted as embedding and passed through bidirectional LSTM to learn from both past and future and takes average of all word vector

In [58]:
vocab_size = min(20000, len(tokenizer.word_index) + 1)
embed_dim  = 128
hidden_dim = 128
num_classes = 4

inputs = Input(shape=(max_len,))
x = Embedding(input_dim=vocab_size, output_dim=embed_dim, mask_zero=True)(inputs)
x = Bidirectional(LSTM(hidden_dim, return_sequences=True))(x)
x = GlobalAveragePooling1D()(x)

outputs = Dense(num_classes, activation="softmax")(x)

model = Model(inputs, outputs)

In [59]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=10,
    batch_size=32,
    class_weight=class_weights,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=2,
            restore_best_weights=True
        )
    ]
)


Epoch 1/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 157s 2s/step - accuracy: 0.5874 - loss: 0.8484 - val_accuracy: 0.6974 - val_loss: 0.5075
Epoch 2/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 167s 2s/step - accuracy: 0.7682 - loss: 0.4479 - val_accuracy: 0.7184 - val_loss: 0.4834
Epoch 3/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 157s 2s/step - accuracy: 0.8583 - loss: 0.3108 - val_accuracy: 0.7526 - val_loss: 0.4987
Epoch 4/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 156s 2s/step - accuracy: 0.9152 - loss: 0.1992 - val_accuracy: 0.7474 - val_loss: 0.5999


In [60]:
y_pred = model.predict(X_test_pad).argmax(axis=1)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 327ms/step
              precision    recall  f1-score   support

           0       0.69      0.52      0.59       120
           1       0.66      0.90      0.76        21
           2       0.98      0.94      0.96       120
           3       0.63      0.77      0.69       120

    accuracy                           0.75       381
   macro avg       0.74      0.78      0.75       381
weighted avg       0.76      0.75      0.75       381

[[ 62   2   1  55]
 [  1  19   1   0]
 [  1   6 113   0]
 [ 26   2   0  92]]


__ATTENTION MODEL__

Attention is added to make the model focus on needed words unlike basleline model. First the vector of importance created then made to one score. Assign high negative number for padding to get 0 while passing through attantion 

In [61]:
class Attention(tf.keras.layers.Layer):
    def __init__(self, hidden_dim):
        super().__init__()
        self.W = Dense(hidden_dim)
        self.v = Dense(1)

    def call(self, inputs, mask=None):
        # inputs: (batch, time, hidden)
        score = tf.nn.tanh(self.W(inputs))     
        score = self.v(score)                 
        score = tf.squeeze(score, axis=-1)    

        if mask is not None:
            mask = tf.cast(mask, tf.float32)
            score += (1.0 - mask) * -1e9      

        weights = tf.nn.softmax(score, axis=1)
        context = tf.reduce_sum(
            inputs * tf.expand_dims(weights, -1),
            axis=1
        )
        return context

In [62]:
vocab_size  = min(20000, len(tokenizer.word_index) + 1)
embed_dim   = 128
hidden_dim  = 128
num_classes = 4
inputs = Input(shape=(max_len,))

x = Embedding(
    input_dim=vocab_size,
    output_dim=embed_dim,
    mask_zero=True
)(inputs)

mask = x._keras_mask

x = Bidirectional(LSTM(hidden_dim, return_sequences=True, dropout=0.3))(x)
x = Attention(hidden_dim)(x, mask=mask)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation="softmax")(x)

attn_model = Model(inputs, outputs)


C:\Users\sriva\miniconda3\Lib\site-packages\keras\src\layers\layer.py:982: UserWarning: Layer 'attention_3' (of type Attention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [63]:
attn_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
history = attn_model.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=20,
    batch_size=32,
    class_weight=class_weights,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=4,
            restore_best_weights=True
        )
    ]
)


Epoch 1/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 185s 2s/step - accuracy: 0.5690 - loss: 0.9550 - val_accuracy: 0.6789 - val_loss: 0.6375
Epoch 2/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 191s 2s/step - accuracy: 0.7104 - loss: 0.5877 - val_accuracy: 0.7421 - val_loss: 0.5380
Epoch 3/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 180s 2s/step - accuracy: 0.8008 - loss: 0.4442 - val_accuracy: 0.7553 - val_loss: 0.5194
Epoch 4/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 197s 2s/step - accuracy: 0.8764 - loss: 0.2904 - val_accuracy: 0.7395 - val_loss: 0.5258
Epoch 5/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 189s 2s/step - accuracy: 0.9211 - loss: 0.2121 - val_accuracy: 0.7368 - val_loss: 0.6654
Epoch 6/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 192s 2s/step - accuracy: 0.9369 - loss: 0.1609 - val_accuracy: 0.7263 - val_loss: 0.6664
Epoch 7/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 185s 2s/step - accuracy: 0.9254 - loss: 0.1829 - val_accuracy: 0.7263 - val_loss: 0.6987


In [64]:
y_pred = np.argmax(attn_model.predict(X_test_pad), axis=1)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 345ms/step
              precision    recall  f1-score   support

           0       0.65      0.68      0.66       120
           1       0.75      0.57      0.65        21
           2       0.94      0.97      0.95       120
           3       0.67      0.64      0.66       120

    accuracy                           0.75       381
   macro avg       0.75      0.72      0.73       381
weighted avg       0.75      0.75      0.75       381

[[ 82   1   0  37]
 [  2  12   7   0]
 [  0   3 116   1]
 [ 43   0   0  77]]
